# Table of Contents

## 1. Data Collection and Scaling
### This section is code to obtain TIF Files from Google Earth Engine (GEE) and scale the images to get images where the pixel value is the temperature in Celcius.

### 1.1 Obtain TIF Files in a Loop
##### In this program the code uses an API from GEE to open the data set and obtain an image with the given start and end data and with the given longitude and latitude.

### 1.2 Generate CSV File with min/max Temperatures & Pixel Values
#### This generates a CSV File that contains the minimum and maximum temperature and pixel values in that image from GEE.

### 1.3 Masking and Interpolation
#### Using the CSV File from 1.2, the images are masked so that pixel values under the minimum pixel value become NaN. The remaining pixels are interpolated and new images are produced where the pixel values are the temperature value at that point.


## 2. Average Image
#### Using the mean average formula, the average image is found by calculating the average temperature (i.e. pixel value) at that point over the range (Jan 1 2013 - Dec 31 2023). There are 3 types of averages:
#### - Daily: This creates an average based on the day (e.g. Jan-1-2013, Jan-1-2014, Jan-1-2015 etc.)
#### - Monthly: This average is based on the pixel values for the complete month form each year
#### - Annual (Overall): This average is computed using the entire years data (i.e. every image, therefore it is also called the overall average) 

#### 2.1 Daily Average Image
#### 2.2 Monthly Average Image
#### 2.3 Annual Average Image (Overall Average)
#### 2.4 Average Image Temperature Distribution


## 3. Sobel Gradient
#### The Sobel Gradient is a method of finding the gradient with respect to its adjacent and diagonally connected neighbours. There is a vertical and horizontal operator, and the magnitude of the gradient is calculated. The direction can also be calculated, however this would require a different look into the gradient field of temperature.


## 4. Thresholding
#### The threhsolding program uses a simple binarization code where the NaN values are assigned to be red to indicate the ocean, and make the map more clear.


## 5. Linear Regression Prediction
#### Using Linear Regression, the temperature values for each day is predicted.

### 5.1 Generate Regression Prediction Images
#### Induvidual pixels are predicted using the pixels on the same day over the year range (2013-2023). Using the np.polyfit method, a linear curve is generated, which is used ot predict the next data point, or the next year, that being 2024.

### 5.2 Calculate RMSE
#### The Root Mean Square Error (RMSE) for each day is calculated. In this instance it was calucalted for each day in January since it showed high variability in the average and gradient images, yielding a theoretically higher uncertainty than the hotter months.

## 1. Data Collection and Scalin

### 1.1 Obtain TIF Files in a Loop

In [ ]:
import ee
ee.Authenticate()

In [ ]:
import ee
import datetime
import os

# Initialize Earth Engine
ee.Initialize()

# Define a region of interest (ROI)
roi = ee.Geometry.Polygon(
    [[[50.717442, 24.455254],
      [51.693314, 24.455254],
      [50.717442, 26.226590],
      [51.693314, 26.226590]]])

# Define the time range for 4 years
start_date = '2017-01-01'
end_date = '2017-01-02'

# Convert start and end dates to datetime objects
start_datetime = datetime.datetime.strptime(start_date, '%Y-%m-%d')
end_datetime = datetime.datetime.strptime(end_date, '%Y-%m-%d')

# Specify the output root folder
output_root_folder = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Interpolated LST\01-01" # Change this to your desired output root folder

# Iterate over each day in the specified period
current_date = start_datetime
while current_date < end_datetime:
    # Convert the current date to string
    current_date_str = current_date.strftime('%Y-%m-%d')
    
    # Create a folder for the current month and day
    output_folder = os.path.join(output_root_folder, current_date.strftime('%m-%d'))
    os.makedirs(output_folder, exist_ok=True)
    
    # Sentinel-5P dataset
    collection = (ee.ImageCollection("MODIS/061/MOD11A1")
                  .filterBounds(roi)
                  .filterDate(ee.Date(current_date_str), ee.Date(current_date_str).advance(1, 'day'))
                  .select('LST_Day_1km'))

    # Get the mean image
    image = collection.median()

    # Get the minimum and maximum values for the specified region
    stats = image.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=roi,
        scale=1000  # Adjust the scale based on your data
    )

    # Extract min and max values from the statistics for the 'LST_Day_1km' band
    min_value = stats.get('LST_Day_1km_min')
    max_value = stats.get('LST_Day_1km_max')

    # Check if min and max values are not None before using them
    if min_value is not None and max_value is not None:
        try:
            min_value_scaled = (min_value.getInfo() * 0.02) - 273.15
            max_value_scaled = (max_value.getInfo() * 0.02) - 273.15
            print(f"Image {image.id()} - Minimum temperature (Scaled): {min_value_scaled}, Maximum temperature (Scaled): {max_value_scaled}")

            # Get the URL of the image thumbnail with the specified range
            thumbnail_url = image.getThumbURL({
                'region': roi,
                'min': min_value_scaled,
                'max': max_value_scaled,
            })

            # Define the output path with the year as the name
            # output_path = os.path.join(output_folder, current_date.strftime('%Y.tif'))
            output_path = r'C:\Users\Nathan Braganza\Desktop\QEERI_new\Interpolated LST\01-01\2017_unscaled'

            # Download the thumbnail and save it to the local folder
            geemap.ee_export_image(image, filename=output_path, region=roi, scale=1000)
        except Exception as e:
            print(f"Error processing {current_date_str}: {e}")
    
    # Move to the next day
    current_date += datetime.timedelta(days=1)


### 1.2 Generate CSV File with min/max Temperatures & Pixel Values

In [ ]:
import ee
import datetime
import pandas as pd
import numpy as np

# Initialize Earth Engine
ee.Initialize()

# Define a region of interest (ROI)
roi = ee.Geometry.Polygon(
    [[[50.717442, 24.455254],
      [51.693314, 24.455254],
      [50.717442, 26.226590],
      [51.693314, 26.226590]]])

# Specify the date range
start_date_str = '2024-01-01'
end_date_str = '2024-03-01'

# Convert start and end dates to datetime objects
start_date = datetime.datetime.strptime(start_date_str, '%Y-%m-%d')
end_date = datetime.datetime.strptime(end_date_str, '%Y-%m-%d')

# Create an empty list to store the results
temperature_data = []

# Iterate over each date in the range
current_date = start_date
while current_date <= end_date:
    # Convert current date to string format
    current_date_str = current_date.strftime('%Y-%m-%d')

    # Sentinel-5P dataset
    collection = (ee.ImageCollection("MODIS/061/MOD11A1")
                  .filterBounds(roi)
                  .filterDate(ee.Date(current_date_str), ee.Date(current_date_str).advance(1, 'day'))
                  .select('LST_Day_1km'))

    # Get the mean image
    image = collection.median()

    # Get the minimum and maximum values for the specified region
    stats = image.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=roi,
        scale=1000  # Adjust the scale based on your data
    )
    
    # Convert the stats dictionary to a Python dictionary
    stats_info = stats.getInfo()
    
    # Check if the statistics dictionary contains the expected keys
    if 'LST_Day_1km_min' in stats_info and 'LST_Day_1km_max' in stats_info:
        # Extract min and max values from the statistics for the 'LST_Day_1km' band
        min_value = stats_info.get('LST_Day_1km_min')
        max_value = stats_info.get('LST_Day_1km_max')
    
        # Replace None with NaN
        min_value = min_value if min_value is not None else np.nan
        max_value = max_value if max_value is not None else np.nan
    
        # Check if min and max values are valid numbers
        if not np.isnan(min_value) and not np.isnan(max_value):
            # Convert values if needed
            min_value_scaled = (min_value * 0.02) - 273.15
            max_value_scaled = (max_value * 0.02) - 273.15
    
            # Append the temperature data to the list
            temperature_data.append({'Date': current_date_str,
                                     'Min Temperature (Scaled)': min_value_scaled,
                                     'Max Temperature (Scaled)': max_value_scaled,
                                     'Min Pixel Value': min_value,
                                     'Max Pixel Value': max_value})
        else:
            # Skip this date if either min_value or max_value is None
            print(f"No data available for {current_date_str}")
    else:
        # Skip this date if statistics for the 'LST_Day_1km' band are not available
        print(f"No statistics available for {current_date_str}")

    # Move to the next day
    current_date += datetime.timedelta(days=1)

# Convert the list of dictionaries to a DataFrame
temperature_df = pd.DataFrame(temperature_data)

# Save the DataFrame to a CSV file if there's data
if not temperature_df.empty:
    output_csv_path = 'temperature_data_prediction.csv'
    temperature_df.to_csv(output_csv_path, index=False)
    print(f"Temperature data saved to {output_csv_path}")
else:
    print("No temperature data available.")

# Display the collected temperature data
print(temperature_df)


### 1.3 Masking & Interpolation

In [ ]:
# For one subdirectory

import os
import rasterio
import numpy as np
import pandas as pd

def find_date_from_path(tiff_path):
    """Extract date from the subdirectory and file name."""
    subdirectory_date = os.path.basename(os.path.dirname(tiff_path))  # Extract date from subdirectory (MM-DD)
    file_date = os.path.splitext(os.path.basename(tiff_path))[0]  # Extract date from file name (YYYY)
    return f"{subdirectory_date}-{file_date}"  # Concatenate dates in the format MM-DD-YYYY

def read_temperature_data(csv_file):
    """Read temperature data from the CSV file."""
    temperature_data = {}
    df = pd.read_csv(csv_file)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')  # Adjust the date format
    for _, row in df.iterrows():
        date = row['Date'].strftime('%m-%d-%Y')  # Format date as MM-DD-YYYY
        temperature_data[date] = {
            'min_temp': row['Min Temperature (Scaled)'],
            'max_temp': row['Max Temperature (Scaled)'],
            'min_pixel_value': row['Min Pixel Value'],
            'max_pixel_value': row['Max Pixel Value']
        }
    return temperature_data


def interpolate_tiff(tiff_path, temperature_data, output_dir):
    # Extract date from the subdirectory and file name
    date = find_date_from_path(tiff_path)
    # Check if the date exists in the temperature data
    if date in temperature_data:
        min_temp = temperature_data[date]['min_temp']
        max_temp = temperature_data[date]['max_temp']
        min_pixel_value = temperature_data[date]['min_pixel_value']
        max_pixel_value = temperature_data[date]['max_pixel_value']
        # Open the GeoTIFF file
        with rasterio.open(tiff_path) as src:
            # Read the raster data
            data = src.read(1)
            # Apply mask to remove pixels below the minimum pixel value
            masked_data = np.where(data < min_pixel_value, np.nan, data)
            # Interpolate the masked data using temperature range
            interpolated_data = (masked_data - min_pixel_value) * (max_temp - min_temp) / (max_pixel_value - min_pixel_value) + min_temp
            # Save the interpolated raster image
            output_file_path = os.path.join(output_dir, f"{os.path.basename(tiff_path)[:-4]}_interpolated.tif")
            with rasterio.open(output_file_path, 'w', **src.profile) as dst:
                dst.write(interpolated_data, 1)
            print(f"Interpolated image saved as {output_file_path}")
    else:
        print(f"No temperature data found for date {date}")

# Paths
input_dir = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Real 2024 Data (Jan1-Mar1)\02-29"
output_dir = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Real 2024 Data (Jan1-Mar1)\02-29"
csv_file = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Programs\temperature_data_prediction.csv"

# Read temperature data from the CSV file
temperature_data = read_temperature_data(csv_file)

# Iterate over each TIFF file in the input directory
for root, _, files in os.walk(input_dir):
    for file in files:
        if file.endswith(".tif"):
            tiff_path = os.path.join(root, file)
            interpolate_tiff(tiff_path, temperature_data, output_dir)


In [ ]:
# For loop to run interploate function for all subdirectories
import os

# Define input and output directories
input_root_dir = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Real 2024 Data (Jan1-Mar1)"
output_root_dir = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Real 2024 Data (Jan1-Mar1)"

# Function to create output directory structure based on input directory structure
def create_output_dir(input_dir, output_root_dir):
    for root, dirs, files in os.walk(input_dir):
        # Create corresponding output directory structure
        rel_path = os.path.relpath(root, input_dir)
        output_dir = os.path.join(output_root_dir, rel_path)
        os.makedirs(output_dir, exist_ok=True)

# Create output directory structure
create_output_dir(input_root_dir, output_root_dir)

# Function to interpolate TIFF files in each subdirectory and save them to corresponding output directories
def interpolate_tiff_in_subdirectories(input_root_dir, output_root_dir, min_temp, max_temp, min_pixel_value, max_pixel_value):
    for root, dirs, files in os.walk(input_root_dir):
        for file in files:
            if file.endswith(".tif"):
                tiff_path = os.path.join(root, file)
                rel_path = os.path.relpath(root, input_root_dir)
                output_dir = os.path.join(output_root_dir, rel_path)
                interpolate_tiff(tiff_path, min_temp, max_temp, min_pixel_value, max_pixel_value, output_dir)

# Call the function to interpolate TIFF files in each subdirectory
interpolate_tiff_in_subdirectories(input_root_dir, output_root_dir, min_temp, max_temp, min_pixel_value, max_pixel_value)

## 2. Average Image

### 2.1 Daily Average Image

In [ ]:
# TIF FILE

# Average Image for processing
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt

def plot_average_image(image_folder, output_file=None, save_as_tiff=False):
    # List all TIFF files in the specified folder
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.tif')]

    # Check if there are TIFF files in the folder
    if not image_files:
        print("No TIFF files found in the specified folder: ", image_folder)
        return

    # Open the first image to get dimensions
    first_image_path = os.path.join(image_folder, image_files[0])
    first_image = tifffile.imread(first_image_path)
    height, width = first_image.shape

    # Initialize an array to store pixel values
    pixel_sum = np.zeros((height, width), dtype=np.float64)
    count = np.zeros((height, width), dtype=np.uint64)

    # Iterate through each image and accumulate pixel values
    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)

        try:
            image = tifffile.imread(image_path).astype(np.float64)
        except Exception as e:
            print(f"Skipping file {image_file} due to error: {e}")
            continue

        # Ensure images have the same dimensions
        if image.shape != (height, width):
            print(f"Skipping {image_file} due to different dimensions.")
            continue

        # Accumulate pixel values
        pixel_sum += np.nan_to_num(image)
        count += ~np.isnan(image)

    # Calculate average pixel values
    average_pixels = np.divide(pixel_sum, count, out=np.zeros_like(pixel_sum), where=(count != 0)).astype(np.uint16)

    # Create a mask to get rid of zero values
    mask = average_pixels != 0

    # Apply the mask to the average image
    masked_average_image = np.where(mask, average_pixels, np.nan)

    # Plot the masked average image
    fig, ax = plt.subplots()
    im = ax.imshow(masked_average_image, cmap='viridis')
    subdir_name = os.path.basename(image_folder)
    # plt.title('Average Image ' + subdir_name, loc='center', fontweight='bold')
    # cbar = fig.colorbar(im, ax=ax)
    # cbar.set_label('Temperature (\u00b0C)', fontweight='bold')
    plt.axis('off')

    # Save the plot if output file is provided
    # if output_file:
    #     plt.savefig(output_file, bbox_inches='tight', pad_inches=0, dpi=plt.gcf().dpi)
    
    # Save the average image as TIFF file if specified
    if save_as_tiff:
        tifffile.imwrite(output_file.replace('.png', '.tif'), masked_average_image)

    plt.show()
    plt.close()

# Example usage: Specify the folder containing TIFF files
folder_path = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Interpolated LST\01-01"
output_file = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Interpolated LST\01-01\Average_Image_01-01.tif"
plot_average_image(folder_path, output_file, save_as_tiff=True)


In [ ]:
# FOR LOOP


import os

# Define the main directory containing subdirectories with images
main_directory = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Interpolated LST"

# Iterate over each subdirectory in the main directory
for subdir in os.listdir(main_directory):
    subdir_path = os.path.join(main_directory, subdir)
    if os.path.isdir(subdir_path):  # Check if it's a directory
        # Define the output file path for the plot
        output_file_path = os.path.join(subdir_path, f'Average_Image_{subdir}.tif')
        
        # Call the plot_average_image function for the current subdirectory
        plot_average_image(subdir_path, output_file=output_file_path, save_as_tiff=True)


In [ ]:
# PLOT W? COLORBAR

import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt

def plot_average_image(image_folder, output_file=None, save_as_tiff=False, dpi=300):
    # List all TIFF files in the specified folder
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.tif')]

    # Check if there are TIFF files in the folder
    if not image_files:
        print("No TIFF files found in the specified folder: ", image_folder)
        return

    # Open the first image to get dimensions
    first_image_path = os.path.join(image_folder, image_files[0])
    first_image = tifffile.imread(first_image_path)
    height, width = first_image.shape

    # Initialize an array to store pixel values
    pixel_sum = np.zeros((height, width), dtype=np.float64)
    count = np.zeros((height, width), dtype=np.uint64)

    # Iterate through each image and accumulate pixel values
    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)

        try:
            image = tifffile.imread(image_path).astype(np.float64)
        except Exception as e:
            print(f"Skipping file {image_file} due to error: {e}")
            continue

        # Ensure images have the same dimensions
        if image.shape != (height, width):
            print(f"Skipping {image_file} due to different dimensions.")
            continue

        # Accumulate pixel values
        pixel_sum += np.nan_to_num(image)
        count += ~np.isnan(image)

    # Calculate average pixel values
    average_pixels = np.divide(pixel_sum, count, out=np.zeros_like(pixel_sum), where=(count != 0)).astype(np.uint16)

    # Create a mask to get rid of zero values
    mask = average_pixels != 0

    # Apply the mask to the average image
    masked_average_image = np.where(mask, average_pixels, np.nan)

    # Plot the masked average image
    fig, ax = plt.subplots()
    im = ax.imshow(masked_average_image, cmap='viridis')
    subdir_name = os.path.basename(image_folder)
    plt.title('Average Image ' + subdir_name, loc='center', fontweight='bold')
    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label('Temperature (\u00b0C)', fontweight='bold')
    plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)
    plt.axis('off')

    # Save the plot if output file is provided
    if output_file:
        plt.savefig(output_file, bbox_inches='tight', pad_inches=0.1, dpi=300)
    
    # # Save the average image as TIFF file if specified
    # if save_as_tiff:
    #     tifffile.imwrite(output_file.replace('.png', '.tif'), masked_average_image)

    plt.show()
    plt.close()

# Example usage: Specify the folder containing TIFF files
folder_path = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Interpolated LST\01-01"
output_file = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Interpolated LST\01-01\Average_Plot.png"
plot_average_image(folder_path, output_file, save_as_tiff=True, dpi=300)


### 2.2 Monthly Average Image

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tifffile

# Define the directory containing the images
directory = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\TIF_Predictions"
output_directory = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Average (Monthly and Overall) - Predicted"

# Function to load and average images for a specific month
def average_images_for_month(directory, month_prefix):
    # Get a list of image files for the given month prefix
    image_files = sorted([f for f in os.listdir(directory) if f.startswith(month_prefix) and f.endswith(".tif")])

    if len(image_files) == 0:
        raise ValueError(f"No TIFF files found for month {month_prefix}")

    # Initialize an empty list to store images for averaging
    images = []

    # Load each image and append it to the list
    for image_file in image_files:
        image_path = os.path.join(directory, image_file)
        image = tifffile.imread(image_path)
        images.append(image)

    # Convert the list of images to a NumPy array
    images = np.array(images)

    # Calculate the average image
    average_image = np.nanmean(images, axis=0)

    if average_image.ndim != 2:
        raise ValueError(f"Invalid shape for average image for month {month_prefix}")

    return average_image

# Plot and save average images for each month
for month_number in range(1, 13):
    month_prefix = f"{month_number:02d}-"
    try:
        average_image = average_images_for_month(directory, month_prefix)
        fig = plt.imshow(average_image, cmap='viridis')
        plt.axis('off')
        plt.title(f'Predicted Average Image {month_number:02d}-2024', fontweight='bold')
        cbar = fig.colorbar(im, ax=ax)
        cbar.set_label('Temperature (\u00b0C)', fontweight='bold')
        # plt.savefig(os.path.join(output_directory, f'{month_number:02d}_average_image.tif'), dpi=300, bbox_inches='tight', pad_inches=0.1)
        plt.show()
    except Exception as e:
        print(f"Error processing month {month_prefix}: {e}")


### 2.3 Annual Average Image (Overall Average)

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

def load_images(directory):
    file_paths = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.tif'):
                file_paths.append(os.path.join(root, file))
    images = [cv2.imread(file_path, cv2.IMREAD_UNCHANGED) for file_path in file_paths]
    return images

def apply_mask(images):
    masked_images = []
    for img in images:
        mask = img == 0
        img[mask] = np.nan
        masked_images.append(img)
    return masked_images

def generate_average_image(images):
    if not images:
        return None  # No images provided
    
    all_nan = all(np.all(np.isnan(img)) for img in images)
    if all_nan:
        return np.full_like(images[0], np.nan)  # All values are NaN, return an array filled with NaNs
    
    valid_images = [img for img in images if np.any(~np.isnan(img))]  # Filter out images with all NaN values
    average_image = np.nanmean(np.array(valid_images), axis=0)
    return average_image


def plot_image(image, output_file):
    fig, ax = plt.subplots()
    im = ax.imshow(image, cmap='viridis')
    subdir_name = os.path.basename(image)
    plt.title('Overall Average Image LST Day', loc='center', fontweight='bold')
    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label('Temperature (\u00b0C)', fontweight='bold')
    plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)
    plt.axis('off')
    plt.savefig(output_file, bbox_inches='tight', dpi=300)
    

if __name__ == "__main__":
    main_directory = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Interpolated LST"
    output_file = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\OverallAverage.png"  # Include the filename
    
    # Load all TIFF images
    images = load_images(main_directory)
    
    # Apply mask
    masked_images = apply_mask(images)
    
    # Generate average image
    average_img = generate_average_image(masked_images)
    
    # Plot average image
    plot_image(average_img, output_file)



### 2.4 Average Image Temperature Distribution

In [ ]:
import os
import numpy as np
import tifffile as tf
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Directory containing the TIFF images
directory = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Average (Monthly and Overall) - Real\Monthly Averages\Distribution Plot"

# List all TIFF files in the directory
tif_file_paths = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.tif')]

# Sort the TIFF file paths by their numeric part
tif_file_paths.sort(key=lambda x: int(os.path.basename(x).split('_')[0]))

# Load and plot histograms for each TIFF image
plt.figure(figsize=(10, 6))

# Get a cyclic colormap for smooth transition
cmap = cm.get_cmap('twilight_shifted')

for i, tif_file_path in enumerate(tif_file_paths):
    # Load the TIFF file
    image_data = tf.imread(tif_file_path)

    # Flatten the image data to 1D array
    flattened_data = image_data.flatten()

    # Calculate color based on the progression index
    color = cmap(i / len(tif_file_paths))

    # Plot the histogram with color
    plt.hist(flattened_data, bins=70, color=color, alpha=0.6, label=os.path.basename(tif_file_path).split('_')[0], density=False)

plt.title('Histogram of Temperature Distribution (Average Image)')
plt.xlabel('Temperature (°C)')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.savefig(r'C:\Users\Nathan Braganza\Desktop\QEERI_new\TempDistribution.png', bbox_inches='tight', dpi=300)  # Use bbox_inches='tight' to remove extra white spaces


plt.show()


## 3. Sobel Gradient

In [ ]:
# DAILY GRADIENT

import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

def calculate_sobel_gradient(image):
    # Calculate the gradient using the Sobel operator
    gradient_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    gradient_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    
    # Compute the magnitude of the gradient
    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    
    return gradient_magnitude

def plot_gradient_image(gradient_image, subdir_name, subdir_path):
    # Plot the gradient image
    fig, ax = plt.subplots()
    im = ax.imshow(gradient_image, cmap='viridis')
    plt.title(f'Sobel Gradient {subdir_name}', loc='center', fontweight='bold')
    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label('Temperature (\u00b0C/m)', fontweight='bold')
    plt.axis('off')
    
    # Save the plot at 300 DPI
    output_file = os.path.join(subdir_path, f'Sobel_Gradient_{subdir_name}.png')
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()



# Example usage: Specify the folder containing TIFF files
folder_path = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Interpolated LST\01-01"
subdir_name = os.path.basename(folder_path)

#Load an image
image_path = os.path.join(folder_path, f"Average_Image_{subdir_name}.tif")
print("Image Path:", image_path)  # Debug statement
image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
if image is None:
    print("Error: Unable to read image from:", image_path)
else:
    # Calculate the Sobel gradient for each pixel in the image
    gradient_image = calculate_sobel_gradient(image)

    # Plot and save the gradient image
    plot_gradient_image(gradient_image, subdir_name, folder_path)


In [ ]:
# DAILY FOR LOOP


import os

# Define the main directory containing subdirectories with images
main_directory = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Interpolated LST"

def process_average_image(subdir_path, subdir_name):
    # Look for an image file starting with "Average_Image_" in the subdirectory
    for file_name in os.listdir(subdir_path):
        if file_name.startswith("Average_Image_") and file_name.endswith(".tif"):
            image_path = os.path.join(subdir_path, file_name)
            print("Processing:", image_path)
            # Load the image
            image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
            if image is not None:
                # Calculate the Sobel gradient
                gradient_image = calculate_sobel_gradient(image)
                # Plot and save the Sobel gradient
                plot_gradient_image(gradient_image, subdir_name, subdir_path)
            else:
                print("Error: Unable to read image from", image_path)

# Iterate over each subdirectory in the main directory
for subdir in os.listdir(main_directory):
    subdir_path = os.path.join(main_directory, subdir)
    if os.path.isdir(subdir_path):
        print("Processing subdirectory:", subdir)
        process_average_image(subdir_path, subdir)


In [ ]:
# MONTHLY GRADIENT


import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

def calculate_sobel_gradient(image):
    # Calculate the gradient using the Sobel operator
    gradient_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    gradient_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    
    # Compute the magnitude of the gradient
    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    
    return gradient_magnitude

def plot_gradient_image(gradient_image, subdir_name, image_name, output_dir):
    # Plot the gradient image
    fig, ax = plt.subplots()
    im = ax.imshow(gradient_image, cmap='viridis')
    plt.title(f'Sobel Gradient {image_name[:2]}', loc='center', fontweight='bold')
    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label('Temperature (\u00b0C/m)', fontweight='bold')
    plt.axis('off')
    
    # Save the plot at 300 DPI
    output_file = os.path.join(output_dir, f'Sobel_Gradient_{image_name[:2]}.png')
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.close()

def process_images_in_directory(folder_path):
    # Iterate over all files in the directory
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.tif'):
            image_path = os.path.join(folder_path, file_name)
            image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
            if image is None:
                print("Error: Unable to read image from:", image_path)
            else:
                subdir_name = os.path.basename(folder_path)
                gradient_image = calculate_sobel_gradient(image)
                plot_gradient_image(gradient_image, subdir_name, os.path.splitext(file_name)[0], folder_path)

# Example usage: Specify the folder containing TIFF files
folder_path = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Average (Monthly and Overall)\Monthly Averages\Distribution Plot"

# Process all images in the directory
process_images_in_directory(folder_path)


In [ ]:
# ANNUAL GRADIENT (OVERALL)

import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tifffile import imread
from skimage.filters import sobel

def calculate_average_image(main_dir):
    # Initialize variables for average image
    avg_image = None
    count = 0
    min_height = float('inf')
    min_width = float('inf')

    # Iterate through subdirectories and process images
    for subdir, _, files in os.walk(main_dir):
        for i, file in enumerate(files):
            if file.endswith('.tif') and not file.startswith('Average') and not file.startswith('prediction'):
                if i % 10 != 0:  # Plot every 10th image
                    continue
                image_path = os.path.join(subdir, file)
                image = imread(image_path).astype(np.float64)
                nan_ratio = np.isnan(image).sum() / image.size
                if nan_ratio > 0.75:  # Skip image if NaN ratio exceeds 80%
                    continue
                if np.isnan(image).all():  # If all values in the image are NaN, skip it
                    continue
                if avg_image is None:
                    avg_image = image
                else:
                    min_height = min(min_height, image.shape[0])
                    min_width = min(min_width, image.shape[1])
                    avg_image = np.nanmean(np.dstack((avg_image[:min_height, :min_width], image[:min_height, :min_width])), axis=2)
                count += 1

    # Check if all images have NaN values or 0, if so, return None
    if count == 0:
        return None

    return avg_image

def calculate_sobel_gradient(image):
    # Calculate the gradient using the Sobel operator
    gradient_x = sobel(image, axis=1)
    gradient_y = sobel(image, axis=0)
    
    # Compute the magnitude of the gradient
    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    
    return gradient_magnitude

def plot_and_save_gradient_image(gradient_image, average_image, output_dir):
    # Plot average image with colorbar
    plt.imshow(average_image, cmap='viridis')
    cbar = plt.colorbar(label='Temperature (\u00b0C)')
    cbar.ax.yaxis.label.set_fontweight('bold')  # Set colorbar label font weight to bold
    plt.title('Average LST Day (2013-2023)')
    plt.axis('off')  # Remove outside plot lines
    
    # Make label bold
    plt.title('Overall Average LST Day (2013-2023)', fontweight='bold')
    
    plt.savefig(os.path.join(output_dir, 'Average_LST_Day.png'), dpi=300, bbox_inches='tight', pad_inches=0)
    plt.show()
    plt.close()

    # Plot gradient image with title and colorbar and save as png
    plt.imshow(gradient_image, cmap='viridis')
    cbar = plt.colorbar()
    cbar.set_label('Temperature (\u00b0C/m)')
    cbar.ax.yaxis.label.set_fontweight('bold')  # Set colorbar label font weight to bold
    plt.title('Sobel Gradient LST Day Average (2013-2023)')
    plt.axis('off')  # Remove outside plot lines
    
    # Make label bold
    plt.title('Overall LST Day Average (2013-2023)', fontweight='bold')
    
    plt.savefig(os.path.join(output_dir, 'Overall_LST_Day_Average.png'), dpi=300, bbox_inches='tight', pad_inches=0)
    plt.show()
    plt.close()

# Example usage
main_directory = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Interpolated LST"
average_image = calculate_average_image(main_directory)
if average_image is not None:
    gradient_image = calculate_sobel_gradient(average_image)
    plot_and_save_gradient_image(gradient_image, average_image, main_directory)
else:
    print("All images contain NaN or 0 values, unable to calculate average.")


## 4. Thresholding

In [ ]:
import os
import tifffile as tf
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

# Define the parent directory where subdirectories contain the TIFF files
parent_directory = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Interpolated LST"

# Iterate over each subdirectory in the parent directory
for subdir, dirs, files in os.walk(parent_directory):
    for filename in files:
        if filename.startswith('Average_Image') and filename.endswith('.tif'):
            # Construct the full path to the TIFF file
            tif_file_path = os.path.join(subdir, filename)
            
            # Load the TIFF image
            image_data = tf.imread(tif_file_path)

            # Define threshold values
            thresholds = [25, 32.1, 45]  # Add or remove threshold values as needed

            # Create subplots for each threshold
            fig, axs = plt.subplots(1, len(thresholds), figsize=(15, 5))
            legend_patches = [mpatches.Patch(color='black', label='Below Standard'),
                              mpatches.Patch(color='white', label='Above Standard'),
                              mpatches.Patch(color='red', label='NaN')]

            for i, threshold in enumerate(thresholds):
                # Initialize an empty binary image (3D array for RGB representation)
                binary_image = np.zeros((image_data.shape[0], image_data.shape[1], 3), dtype=np.uint8)
                white_pixels = 0
                nan_pixels = 0

                # Iterate through each pixel and apply threshold conditions
                for row in range(image_data.shape[0]):
                    for col in range(image_data.shape[1]):
                        value = image_data[row, col]

                        if np.isnan(value):
                            # Set NaN values to red (255, 0, 0)
                            binary_image[row, col] = [255, 0, 0]
                            nan_pixels += 1  # Increment NaN pixel count
                        elif value > threshold:
                            # Set values above threshold to white (255, 255, 255)
                            binary_image[row, col] = [255, 255, 255]
                            white_pixels += 1  # Increment white pixel count
                        else:
                            # Set values below or equal to threshold to black (0, 0, 0)
                            binary_image[row, col] = [0, 0, 0]

                # Calculate total valid pixels (excluding NaN)
                total_valid_pixels = image_data.size - nan_pixels

                # Calculate white pixel ratio (white pixels / total valid pixels)
                white_pixel_ratio = (white_pixels / total_valid_pixels) * 100 if total_valid_pixels > 0 else 0

                # Plot the binary image
                axs[i].imshow(binary_image)
                axs[i].set_title('Threshold = {}°C\nWhite Pixel Ratio: {:.2f}%'.format(threshold, white_pixel_ratio))
                axs[i].axis('off')

            # Add legend outside the subplots
            legend = fig.legend(handles=legend_patches, loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=3, edgecolor='grey', facecolor='lightgrey', frameon=True)

            # Customize the legend font color
            for text in legend.get_texts():
                text.set_color('black')

            # Save the plot with 300 dpi in the same directory as the TIFF file
            save_dir = os.path.dirname(tif_file_path)
            save_path = os.path.join(save_dir, 'TempThresh.png')
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            
            # Close the plot to free up resources
            plt.close(fig)

            print(f"Processed: {filename}, White Pixel Ratio (thresholds {thresholds}): {white_pixel_ratio:.2f}%")

print("Processing complete.")


## 5. Linear Regression Prediction

### 5.1 Generate Regression Prediction Images (Daily)

In [ ]:
import os
import tifffile as tf
import numpy as np
from sklearn.linear_model import LinearRegression

def load_tiff_images(directory):
    """Load all TIFF images from the specified directory."""
    files = os.listdir(directory)
    tiff_files = [file for file in files if file.lower().endswith('.tif') or file.lower().endswith('.tiff')]
    if not tiff_files:
        print("No TIFF files found in the directory.")
        return []
    images = []
    for file in tiff_files:
        file_path = os.path.join(directory, file)
        image = tf.imread(file_path)
        images.append(image)
    return images

def retrieve_pixel_values(images, x, y):
    pixel_values = []
    for image in images:
        if 0 <= x < image.shape[0] and 0 <= y < image.shape[1]:
            pixel_value = image[x, y]
            if np.isscalar(pixel_value) and not np.isnan(pixel_value):
                pixel_values.append(pixel_value)
    return pixel_values

def compute_linear_regression(pixel_values):
    if not pixel_values:
        return np.nan

    X = np.arange(len(pixel_values)).reshape(-1, 1)  # Reshape for scikit-learn
    y = np.array(pixel_values)

    # Fit linear regression model
    model = LinearRegression()
    model.fit(X, y)

    # Predict the value following the last data point
    next_index = len(pixel_values)
    next_value = model.predict([[next_index]])[0]

    return next_value

# Main directory
main_directory = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Interpolated LST"
output_directory = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\TIF_Predictions"

# Iterate over subdirectories
for subdirectory in os.listdir(main_directory):
    subdirectory_path = os.path.join(main_directory, subdirectory)
    if os.path.isdir(subdirectory_path):
        # Exclude 'Average' or 'prediction' TIFF files
        tiff_files = [file for file in os.listdir(subdirectory_path) if file.lower().endswith('.tif') or file.lower().endswith('.tiff')]
        filtered_tiff_files = [file for file in tiff_files if not file.lower().startswith('Average') and not file.lower().startswith('prediction')]
        if filtered_tiff_files:
            images = load_tiff_images(subdirectory_path)
            predicted_values = np.empty(images[0].shape)

            for x in range(images[0].shape[0]):  # Iterate over rows
                for y in range(images[0].shape[1]):  # Iterate over columns
                    pixel_values = retrieve_pixel_values(images, x, y)

                    # Predict the next value
                    next_value = compute_linear_regression(pixel_values)

                    # Store the predicted value in the new array
                    predicted_values[x, y] = next_value

            # Save the predicted values as TIFF file
            output_filename = os.path.join(output_directory, subdirectory + ".tif")
            tf.imwrite(output_filename, predicted_values)
        print('Complete:', subdirectory)

### 5.1 Generate Regression Prediction Images (Monthly)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tf

# Define the directory containing the images
directory = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\TIF_Predictions"
output_directory = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Average (Monthly and Overall) - Predicted"

# Function to load and average images for a specific month
def average_images_for_month(directory, month_prefix):
    # Get a list of image files for the given month prefix
    image_files = sorted([f for f in os.listdir(directory) if f.startswith(month_prefix) and f.endswith(".tif")])

    if len(image_files) == 0:
        raise ValueError(f"No TIFF files found for month {month_prefix}")

    # Initialize an empty list to store images for averaging
    images = []

    # Load each image and append it to the list
    for image_file in image_files:
        image_path = os.path.join(directory, image_file)
        image = tf.imread(image_path)
        images.append(image)

    # Convert the list of images to a NumPy array
    images = np.array(images)

    # Calculate the average image
    average_image = np.nanmean(images, axis=0)

    if average_image.ndim != 2:
        raise ValueError(f"Invalid shape for average image for month {month_prefix}")

    return average_image

# Plot and save average images for each month
for month_number in range(1, 13):
    month_prefix = f"{month_number:02d}-"
    try:
        average_image = average_images_for_month(directory, month_prefix)
        
        # Plot the average image
        plt.figure(figsize=(8, 6))  # Create a new figure
        plt.imshow(average_image, cmap='viridis')
        plt.axis('off')
        plt.title(f'Predicted Average Image {month_number:02d}-2024', fontweight='bold')
        
        # Add colorbar to the plot
        cb = plt.colorbar()
        cb.set_label('Temperature (\u00b0C)', fontweight='bold')
        
        # Save the plot to the specified output directory
        save_path = os.path.join(output_directory, f'{month_number:02d}_average_image.png')
        plt.savefig(save_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
        
        plt.show()  # Display the plot

    except Exception as e:
        print(f"Error processing month {month_prefix}: {e}")

print("Processing complete.")


### 5.1 Generate Regression Prediction Images (Annual)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tf

# Define the directory containing the TIFF files
directory = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\TIF_Predictions"

# Function to load and accumulate all images from the directory
def load_and_accumulate_images(directory):
    # Get a list of all TIFF files in the directory
    image_files = [f for f in os.listdir(directory) if f.endswith(".tif")]

    if not image_files:
        raise ValueError("No TIFF files found in the directory.")

    # Initialize an empty list to store loaded images
    images = []

    # Load each TIFF image and append it to the list
    for image_file in image_files:
        image_path = os.path.join(directory, image_file)
        image = tf.imread(image_path)
        images.append(image)

    # Convert the list of images to a NumPy array
    images = np.array(images)

    return images

# Load and accumulate all images
try:
    images_array = load_and_accumulate_images(directory)

    # Calculate the average image
    if images_array.size > 0:
        average_image = np.nanmean(images_array, axis=0)
    else:
        raise ValueError("No valid images loaded.")

    # Plot the average image
    plt.figure(figsize=(8, 6))
    plt.imshow(average_image, cmap='viridis')
    plt.title('Average Image 2024', fontweight='bold')
    cb = plt.colorbar()
    cb.set_label('Temperature (\u00b0C)', fontweight='bold')
    plt.axis('off')

    # Save the average image
    output_directory = r"C:\Users\Nathan Braganza\Desktop\QEERI_new"
    save_path = os.path.join(output_directory, 'Average_Image_2024.png')
    plt.savefig(save_path, dpi=300, bbox_inches='tight', pad_inches=0.1)

    plt.show()

except Exception as e:
    print(f"Error: {e}")

print("Processing complete.")


In [ ]:
# PLOT PREDICTIONS


import os
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import tifffile as tf

# Define directories
tif_directory = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\TIF_Predictions"
save_directory = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Interpolated LST"

# Iterate over each TIF file in the directory
for filename in os.listdir(tif_directory):
    if filename.endswith('.tif'):
        # Load the TIF image
        tif_file_path = os.path.join(tif_directory, filename)
        image_data = tf.imread(tif_file_path)

        # Plot the image data
        plt.figure(figsize=(8, 6))  # Adjust figure size as needed
        plt.imshow(image_data, cmap='viridis')

        # Customize color bar label
        cb = plt.colorbar(label='Temperature (°C)')

        # Customize color bar label font
        cb.ax.yaxis.label.set_font_properties(FontProperties(weight='bold'))

        # Customize title based on file name
        subfolder_name = os.path.splitext(filename)[0]
        plt.title(f'LST Day Prediction {subfolder_name}-2024', fontweight='bold')

        plt.xlabel('Column Index')
        plt.ylabel('Row Index')

        # Adjust layout and padding
        plt.tight_layout(pad=1.0)  # Adjust padding as needed
        plt.axis('off')

        # Create subdirectory based on TIF file name
        subdirectory_path = os.path.join(save_directory, subfolder_name)
        os.makedirs(subdirectory_path, exist_ok=True)

        # Save the plot to the specified subdirectory with the same name as TIF file
        save_path = os.path.join(subdirectory_path, 'prediction.png')
        plt.savefig(save_path, bbox_inches='tight', dpi=300)  # Use bbox_inches='tight' to remove extra white spaces

        plt.close()  # Close the plot to free up resources
        print('Done:', save_path[-5:])
print("Processing complete.")


### 5.2 Calculate RMSE

In [ ]:
# RMSE
import os
import numpy as np
import tifffile as tf

# Load the second TIFF file
real_tiff_path = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Real 2024 Data (Jan1-Mar1)\01-31\2024_interpolated.tif"
real_image = tf.imread(real_tiff_path)

# Calculate the mean squared error (MSE)
mse = np.nanmean(np.square(predicted_values - real_image))

# Calculate the RMSE by taking the square root of the MSE
rmse = np.sqrt(mse)

print("Root Mean Squared Error (RMSE):", rmse)


## Extra Codes

### Min and Max in Image w/ Location

In [ ]:
import tifffile as tf
import numpy as np
import matplotlib.pyplot as plt

def find_min_max(image_path):
    # Read the TIFF image
    image = tf.imread(image_path)

    # Initialize variables to store min and max values
    min_value = np.inf
    min_location = None
    max_value = -np.inf
    max_location = None

    # Iterate through each pixel
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            # Check if the pixel value is not NaN
            if not np.isnan(image[i, j]):
                # Update min value and location if necessary
                if image[i, j] < min_value:
                    min_value = image[i, j]
                    min_location = (i, j)
                # Update max value and location if necessary
                if image[i, j] > max_value:
                    max_value = image[i, j]
                    max_location = (i, j)

    return min_value, min_location, max_value, max_location

# Example usage:
image_path = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Average (Monthly and Overall)\Monthly Averages\Distribution Plot\05_average_image.tif"  # Replace with the path to your TIFF image
min_val, min_loc, max_val, max_loc = find_min_max(image_path)

# Read the image for plotting
image = tf.imread(image_path)

# Plot the image
plt.imshow(image, cmap='viridis')

# Plot the location of minimum value
plt.plot(min_loc[1], min_loc[0], 'ro', markersize=6, label='Min Value', markeredgewidth=1, markeredgecolor='black')  # Adjust markersize here

# Plot the location of maximum value
plt.plot(max_loc[1], max_loc[0], 'bo', markersize=6, label='Max Value', markeredgewidth=1, markeredgecolor='black')  # Adjust markersize here

# Add legend
plt.legend()

# Show the plot
plt.show()


### Delete Images with a string included

In [ ]:
import os

def delete_average_images(main_directory):
    # Iterate through each file in the main directory
    for root, dirs, files in os.walk(main_directory):
        for file in files:
            # Check if the file name starts with "Average_Image"
            if file.startswith("Temp"):
                # Construct the file path and delete the file
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print(f"Deleted: {file_path}")

# Example usage: Specify the main directory containing the images
main_directory = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Real 2024 Data (Jan1-Mar1)"
delete_average_images(main_directory)


### Circling Bubbbles in Gradient Image

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

def calculate_sobel_gradient(image):
    # Calculate the gradient using the Sobel operator
    gradient_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    gradient_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    
    # Compute the magnitude of the gradient
    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    
    return gradient_magnitude

def plot_gradient_image_with_circles(gradient_image, subdir_name, subdir_path, circle_locations=None):
    # Plot the gradient image
    fig, ax = plt.subplots()
    im = ax.imshow(gradient_image, cmap='viridis')
    plt.title(f'Sobel Gradient {subdir_name}', loc='center', fontweight='bold')
    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label('Temperature (\u00b0C/m)', fontweight='bold')
    plt.axis('off')
    
    # Draw red circles around specified locations
    if circle_locations:
        for location in circle_locations:
            y, x = location  # Assuming location is (row, col) or (y, x)
            # Draw a red circle with radius 10 pixels around each location
            circle = plt.Circle((x, y), radius=4, color='red', fill=False, linewidth=1)
            ax.add_patch(circle)
    
    # Save the plot at 300 DPI
    output_file = os.path.join(subdir_path, f'Sobel_Gradient_{subdir_name}_with_circles.png')
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.close()  # Close the plot to free memory

# Example usage: Specify the folder containing TIFF files
folder_path = r"C:\Users\Nathan Braganza\Desktop\QEERI_new\Interpolated LST\07-25"
subdir_name = os.path.basename(folder_path)

# Load an image
image_path = os.path.join(folder_path, f"Average_Image_{subdir_name}.tif")
print("Image Path:", image_path)  # Debug statement
image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
if image is None:
    print("Error: Unable to read image from:", image_path)
else:
    # Calculate the Sobel gradient for each pixel in the image
    gradient_image = calculate_sobel_gradient(image)
    
    # Define circle locations (example: list of pixel locations)
    circle_locations = [(137,53), (133,40), (138, 93), (123, 68), (111, 54), (50, 63)]
    # circle_locations = [(137, 53), (133, 40), (133,30),(93, 53), (100, 64), (110, 53)
    #                     , (65, 40), (102, 55), (50, 63), (100, 75), (80, 77), (125, 70)]  # List of (y, x) coordinates
    # 1 - Irakaya Farm
    # 2 - Al Karanaa Bird Lagoon
    # 3 - Mesaieed
    # 4 - Abu Nakhla Sewage Ponds
    # 5 - Mazzraty Farm
    # 6 - Dawoodiya Farm
    

    
    # Plot the gradient image with red circles and save the result
    plot_gradient_image_with_circles(gradient_image, subdir_name, folder_path, circle_locations)
    print("Gradient image with circles saved successfully.")
